# 어텐션 메커니즘과 트랜스포머

순환 네트워크의 한 가지 주요 단점은 시퀀스의 모든 단어가 결과에 동일한 영향을 미친다는 것입니다. 이로 인해 Named Entity Recognition 및 기계 번역(Machine Translation)과 같은 시퀀스 작업을 위한 표준 LSTM 인코더-디코더(encoder-decoder) 모델에서 낮은 성능이 발생합니다. 실제로 입력 시퀀스의 특정 단어는 종종 다른 단어보다 순차적 출력에 더 많은 영향을 미칩니다.

기계 번역과 같은 sequence-to-sequence 모델을 고려하십시오. 하나의 네트워크(**encoder**)가 입력 시퀀스를 은닉 상태로 축소하고 다른 네트워크인 **decoder**가 숨김 상태를 변환된 결과로 펼치는 두 개의 반복 네트워크로 구현됩니다. 이 접근 방식의 문제점은 네트워크의 최종 상태가 문장의 시작을 기억하는 데 어려움을 겪어 긴 문장에서 모델의 품질을 저하시킨다는 것입니다.

**어텐션 메커니즘(Attention Mechanisms)** 은 RNN의 각 예측 출력에 대한 각 입력 벡터의 상황별 영향에 가중치를 부여하는 수단을 제공한다. 구현 방법은 입력 RNN과 출력 RNN의 중간 상태 사이에 바로 가기를 만드는 것입니다. 이러한 방식으로 출력 $y_t$를 생성할 때 다른 가중치 계수 $\alpha_{t,i}$.와 함께 모든 입력 은닉 상태 $h_i$를 고려합니다.

![Image showing an encoder/decoder model with an additive attention layer](./images/encoder-decoder-attention.png)
*The encoder-decoder model with additive attention mechanism in [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), cited from [this blog post](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

어텐션 행렬 $\{\alpha_{i,j}\}$은 특정 입력 단어가 출력 시퀀스에서 주어진 단어를 생성할 때 생성하는 데 영향 정도를 나타냅니다. 다음은 이러한 행렬의 예입니다.

![Image showing a sample alignment found by RNNsearch-50, taken from Bahdanau - arviz.org](./images/bahdanau-fig3.png)

*Figure taken from [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

어텐션 메커니즘은 자연어 처리에서 현재 최신 기술을 담당합니다. 그러나 어텐션 메커니즘을 추가하는 것은 RNN의 스케일링 문제로 이어져 모델 파라미터의 수가 크게 증가합니다. RNN 확장의 주요 제약은 모델의 반복적인 특성으로 인해 훈련을 일괄 처리하고 병렬화하기가 어렵다는 것입니다. RNN에서 시퀀스의 각 요소는 순차적으로 처리되어야하므로 쉽게 병렬화 할 수 없습니다.

이 한계점과 결합된 어텐션 메커니즘의 채택으로 오늘날 우리가 알고 있고 사용하는 최첨단 트랜스포머(Transformer) 모델이 BERT에서 OpenGPT3에 이르기까지 탄생했습니다.


## 트랜스포머(Transformer) 모델

각 이전 예측의 컨텍스트를 다음 평가 스텝으로 전달하는 대신 **트랜스포머 모델**은 **위치 인코딩(positional encodings)** 과 어텐션을 사용하여 제공된 텍스트 윈도우에서 주어진 입력의 컨텍스트를 캡처합니다. 아래 이미지는 어텐션이 있는 위치 인코딩이 주어진 윈도우 내에서 컨텍스트를 캡처하는 방법을 보여줍니다.

![Animated GIF showing how the evaluations are performed in transformer models.](./images/transformer-animated-explanation.gif) 

각 입력 포지션이 각 출력 포지션에 독립적으로 매핑되기 때문에 트랜스포머는 RNN보다 더 잘 병렬화(parallelize)할 수 있으므로 훨씬 더 크고 표현력이 풍부한 언어 모델이 가능합니다. 각각의 어텐션 헤드(attention head)는 다운 스트림 자연어 처리 작업을 개선하고 단어 간의 다른 관계를 학습하는 데 사용할 수 있습니다.

**BERT** (Bidirectional Encoder Representations from Transformers) 는 매우 큰 다층 트랜스포머 네트워크입니다(12 layers for *BERT-base*, and 24 for *BERT-large*). 이 모델은 먼저 비지도 학습(문장에서 마스킹 된 단어 예측)을 사용하여 대규모 텍스트 데이터 (위키피디아 + 책)에 대해 사전 학습(pre-training)됩니다. 사전 학습 중에 모델은 상당한 수준의 언어 이해를 흡수한 다음 미세조정(fine-tuning)을 사용하여 다른 데이터셋과 함께 활용할 수 있습니다. 이 과정을 **전이학습(transfer learning)** 이라고 합니다.

![picture from http://jalammar.github.io/illustrated-bert/](./images/jalammarBERT-language-modeling-masked-lm.png)

BERT, DistilBERT를 포함한 트랜스포머 아키텍처에는 다양한 변형이 있습니다. BigBird, OpenGPT3 등은 fine-tuning 할 수 있습니다. [HuggingFace package](https://github.com/huggingface/)는 PyTorch로 이러한 많은 아키텍처를 학습 할 수있는 레파지토리를 제공합니다.

## BERT를 사용하여 텍스트 분류

기존 작업 인 시퀀스 분류를 해결하기 위해 사전 훈련된 BERT 모델을 사용하는 방법을 살펴 보겠습니다. 원래의 AG News 데이터셋을 분류합니다.

먼저 HuggingFace 라이브러리와 데이터셋을 로드하겠습니다:

In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


사전 학습된 BERT 모델을 사용할 것이므로 특정 tokenizer를 사용해야 합니다. 먼저 사전 훈련된 BERT 모델과 관련된 tokenizer를 로드합니다.

HuggingFace 라이브러리에는 `from_pretrained` 함수에 대한 인자(arguments)로 이름을 지정하기 만하면 사용할 수있는 사전 학습된 모델의 저장소가 있습니다. 모델에 필요한 모든 바이너리 파일이 자동으로 다운로드됩니다.

그러나 특정 시간에는 자체 모델을 로드해야 합니다.이 경우 tokenizer의 파라미터, 모델 매개 변수가있는`config.json` 파일, 이진 가중치 등을 포함하여 모든 관련 파일이 포함 된 디렉토리를 지정할 수 있습니다.

그러나 어떤 경우에는 자신의 모델을 로드해야 합니다. 이 경우 모델 파라미터, 이진 가중치 등을 포함한 tokenizer, `config.json` 파일 등의 모든 관련 파일을 포함하는 디렉토리를 지정할 수 있습니다.

In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` 객체에는 텍스트를 인코딩하는 데 직접 사용할 수있는`encode` 함수가 포함되어 있습니다:

In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

그런 다음 훈련 중에 데이터에 액세스하는 데 사용될 iterators를 생성해 보겠습니다. BERT는 자체 인코딩 함수를 사용하기 때문에 이전에 정의한 `padify`와 유사한 패딩 함수를 정의해야 합니다:

In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

우리의 경우 `bert-base-uncased`라는 사전 학습된 BERT 모델을 사용합니다. `BertForSequenceClassfication`패키지를 사용하여 모델을 로드 해보겠습니다. 이렇게 하면 모델에 최종 분류기를 포함하여 분류에 필요한 아키텍처가 이미 있습니다. 최종 분류기의 가중치가 초기화되지 않았고 모델에 사전 학습이 필요하다는 경고 메시지가 표시됩니다. 이는 우리가 수행하려는 작업이기 때문에 완벽하게 괜찮습니다!

In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

이제 훈련을 시작할 준비가 되었습니다! BERT는 이미 사전 훈련되었기 때문에 초기 가중치를 파괴하지 않기 위해 다소 작은 학습률로 시작하고 싶습니다.

모든 힘든 작업은 `BertForSequenceClassification` 모델에 의해 이루어집니다. 학습 데이터에서 모델을 호출하면 입력 미니배치에 대한 손실과 네트워크 출력을 모두 리턴합니다. 파라미터 최적화를 위해 손실을 사용하고 (`loss.backward ()`는 역방향 패스를 수행함), 획득 한 레이블 `labs` (`argmax`를 사용하여 계산 됨)를 예상되는`labels`와 비교하여 훈련 정확도를 계산하기 위해 `out`을 사용합니다.

프로세스를 제어하기 위해 여러 반복에 걸쳐 손실과 정확도를 축적하고 `report_freq` 훈련 주기마다 이를 출력합니다.

이 훈련은 꽤 오랜 시간이 걸릴 수 있으므로 반복 횟수를 제한합니다.

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


BERT 분류가 꽤 좋은 정확도를 제공한다는 것을 알 수 있습니다(특히, 반복 횟수를 늘리고 충분히 오래 기다리면). BERT는 이미 언어의 구조를 꽤 잘 이해하고 있기 때문에 최종 분류자만 fine-tuning하면 됩니다. 그러나 BERT는 큰 모델이기 때문에 전체 학습 프로세스에 오랜 시간이 걸리고 상당한 계산 능력이 필요합니다!(GPU, 바람직하게는 둘 이상).

> **참조:** 이 예에서는 가장 작은 사전 학습된 BERT 모델 중 하나를 사용했습니다. 더 나은 결과를 얻을 수있는 더 큰 모델이 있습니다.

## 모델 성능 평가하기

이제 테스트 데이터셋에서 모델의 성능을 평가할 수 있습니다. 평가 루프는 학습 루프와 매우 유사하지만 `model.eval ()`을 호출하여 모델을 평가 모드로 전환하는 것을 잊지 말아야합니다.

In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Takeaway

이 단원에서는 **transformers** 라이브러리에서 사전 학습된 언어 모델을 가져와 텍스트 분류 작업에 적용하는 것이 얼마나 쉬운지 살펴 보았습니다. 마찬가지로 BERT 모델은 엔티티 추출, 질문 응답 및 기타 NLP 작업에 사용할 수 있습니다.

트랜스포머 모델은 NLP의 최신 최신 기술을 나타내며 대부분의 경우 커스텀 NLP 솔루션을 구현할 때 실험을 시작하는 첫 번째 솔루션이어야합니다. 그러나 고급 신경 모델을 구축하려는 경우 이 모듈에서 설명하는 순환 신경망의 기본 기본 원리를 이해하는 것이 매우 중요합니다.